In [1]:
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from popgym.envs import labyrinth_escape, labyrinth_explore
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:

num_of_cycles = 2
total_timesteps_per_cycle = 5000 
max_steps_per_episode = 5000 

In [3]:

ray.init()

2023-11-26 12:18:48,635	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.9.18
Ray version:,2.3.0


In [4]:
envs = ["LabyrinthEscapeEasy", "LabyrinthEscapeMedium", "LabyrinthEscapeHard", "LabyrinthExploreEasy", "LabyrinthExploreMedium", "LabyrinthExploreHard"]
# envs = ["LabyrinthEscapeEasy"]
ray.tune.registry.register_env("LabyrinthEscapeEasy", lambda env_config: labyrinth_escape.LabyrinthEscapeEasy())
ray.tune.registry.register_env("LabyrinthEscapeMedium", lambda env_config: labyrinth_escape.LabyrinthEscapeMedium())
ray.tune.registry.register_env("LabyrinthEscapeHard", lambda env_config: labyrinth_escape.LabyrinthEscapeHard())
ray.tune.registry.register_env("LabyrinthExploreEasy", lambda env_config: labyrinth_explore.LabyrinthExploreEasy())
ray.tune.registry.register_env("LabyrinthExploreMedium", lambda env_config: labyrinth_explore.LabyrinthExploreMedium())
ray.tune.registry.register_env("LabyrinthExploreHard", lambda env_config: labyrinth_explore.LabyrinthExploreHard())

In [74]:
#Transformer wide -  params 693957
model = {"use_attention": True,
         "max_seq_len":100,
         # "custom_model_config":{
         'attention_num_transformer_units':2,
         'attention_dim':64,
         'attention_num_heads':20,
         # 'memory_tau':50
        # }
        }

# Transforer Length - params 694373
# model = {"use_attention": True,
#          "max_seq_len":100,
#          # "custom_model_config":{
#          'attention_num_transformer_units':7,
#          'attention_dim':64,
#          'attention_num_heads':2,
#          # 'memory_tau':50
#         # }
#         }

# LSTM Default - params 685317

# model = {"use_lstm": True,
#         "max_seq_len":100,
#         'lstm_cell_size': 256 
#         }

In [75]:

mean_reward_per_episode = {}
timesteps_done = {}
for env in envs:
    mean_reward_per_episode[env] = []
    timesteps_done[env] = []

previous_checkpoint_path = None

In [76]:
for cycle_count in range(num_of_cycles):
    for env in envs:
        print(f'Starting Cycle {cycle_count} Environment {env}:')
        config = {
            "env": env,
            "num_gpus": 0,
            "model": model,
            "framework": "torch",
            "env_config": {
                "horizon": max_steps_per_episode
            }
        }

        trainer = PPOTrainer(env=env, config=config)   
        weights = trainer.get_weights()
        # with open('ray_torch_model.pkl', 'wb') as f:
        #         pickle.dump(weights, f)
        # print('Done')
        
        # break
        if previous_checkpoint_path is not None:
            trainer.restore(previous_checkpoint_path+'/'+sorted(os.listdir(previous_checkpoint_path))[-1])

        previous_checkpoint_path = f"saved_checkpoints/agent_cycle_{cycle_count}_env_{env}"
        timesteps_total = 0
        while timesteps_total < total_timesteps_per_cycle:
            weights = trainer.get_weights()
            with open('ray_torch_model.pkl', 'wb') as f:
                pickle.dump(weights, f)
            result = trainer.train()
            mean_reward_per_episode[env].append(result['episode_reward_mean'])
            print(timesteps_total)
            print(result['episode_reward_mean'])
            timesteps_done[env].append(result['timesteps_total'])
            timesteps_total = result['timesteps_total']
            trainer.save(previous_checkpoint_path)

2023-11-26 12:33:17,264	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='LabyrinthEscapeEasy', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('LabyrinthEscapeEasy').build()` instead. This will raise an error in the future!


Starting Cycle 0 Environment LabyrinthEscapeEasy:


(RolloutWorker pid=7860) 2023-11-26 12:33:30,105	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=7860) 2023-11-26 12:33:30,106	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2023-11-26 12:33:31,609	INFO trainable.py:172 -- Trainable.setup took 14.321 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-11-26 12:33:31,609	WARNING util.py:67 -- Install gputil for GPU system monitoring.


Done


In [77]:
ray.shutdown()

In [78]:
with open('ray_torch_model.pkl', 'rb') as f:
    x = pickle.load(f)
sum = 0
for i in x:
    for j in x[i]:
        sum+=x[i][j].size
        # print(x[i][j].shape[0])
        # for k in x[i][j]:
        #     print(k.shape)
print(sum)

693957


In [ ]:
685317
238437

In [98]:
policy = trainer.get_policy()
policy.model

FullyConnectedNetwork_as_AttentionWrapper(
  (_hidden_layers): Sequential(
    (0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=50, out_features=256, bias=True)
        (1): Tanh()
      )
    )
    (1): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): Tanh()
      )
    )
  )
  (_value_branch_separate): Sequential(
    (0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=50, out_features=256, bias=True)
        (1): Tanh()
      )
    )
    (1): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): Tanh()
      )
    )
  )
  (_value_branch): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=64, out_features=1, bias=True)
    )
  )
  (gtrxl): GTrXLNet(
    (linear_layer): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=64, bias=True)
      )
    )
    (attention_laye

In [99]:
import tensorflow as tf
config = {
            "env": env,
            "num_gpus": 0,
            "model": model,
            "framework": "torch",
            "env_config": {
                "horizon": max_steps_per_episode
            }
        }

trainer = PPOTrainer(env=env, config=config)

ModuleNotFoundError: No module named 'tensorflow'